# 0. 패키지 불러오기

In [2]:
!pip install catboost
!pip install category_encoders
!pip install optuna
!pip install kmodes

In [3]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm # matplotlib의 font_manager에서 폰트설정을 변경하여 matplotlib에서도 한글 출력을 가능하게 한다.
import seaborn as sns

from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import * # KFold, cross_val_score, train_test_split
from sklearn.metrics import *  # make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import * # LabelEncoder, StandardScaler
from sklearn.svm import * # SVC, SVR
from sklearn.inspection import *
from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import * # PCA

from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from pathlib import Path

from lightgbm import *
import lightgbm as lgb
import xgboost as xgb
from datetime import datetime
import calendar
import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. 데이터 불러오기

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!ls "/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData"

 cat_part_test.csv    lgbm_test.csv		    submission_stk_lgbm_wj.csv
 cat_test.csv	      lgbm_val.csv		    tab_part_test.csv
 cat_val.csv	      참가자_제공_레이아웃.pdf	    tab_test.csv
 속성_D_코드.csv      rf_part_test.csv		    tab_val.csv
 etc_part_test.csv    rf_test.csv		   'test2_수정 전.csv'
 etc_test.csv	      rf_val.csv		    test2.csv
 etc_val.csv	      sample_submission.csv	    test.csv
 속성_H_코드.csv      submission_cat2.csv	   'train2_수정 전.csv'
 hgbm_part_test.csv   submission_cat.csv	    train2.csv
 hgbm_test.csv	      submission_lgbm.csv	    train.csv
 hgbm_val.csv	      submission_stk_cat_wj2.csv    xgb_part_test.csv
 속성_L_코드.csv      submission_stk_cat_wj.csv     xgb_test.csv
 lgbm_part_test.csv   submission_stk_lgbm_wj2.csv   xgb_val.csv


In [17]:
Rdata_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/train2.csv')
Rdata_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/test2.csv')
Rdata_D = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_D_코드.csv', index_col = 0).T.to_dict()
Rdata_H = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_H_코드.csv', index_col = 0).T.to_dict()
Rdata_L = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/RawData/속성_L_코드.csv', index_col = 0).T.to_dict()
print(Rdata_train.shape)
print(Rdata_train.size)
print(Rdata_test.shape)
print(Rdata_test.size)
Rdata_train.head()

(501951, 99)
49693149
(46404, 98)
4547592


,Unnamed: 0,id,person_attribute_a,person_attribute_a_under,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_under,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_세,person_prefer_d_1_소,person_prefer_d_1_중,person_prefer_d_1_대,person_prefer_d_2_세,person_prefer_d_2_소,person_prefer_d_2_중,person_prefer_d_2_대,person_prefer_d_3_세,person_prefer_d_3_소,person_prefer_d_3_중,...,d1_sm_match_yn,d1_sub_match_yn,d1_all_match_yn,d2_la_match_yn,d2_mi_match_yn,d2_sm_match_yn,d2_sub_match_yn,d2_all_match_yn,d3_la_match_yn,d3_mi_match_yn,d3_sm_match_yn,d3_sub_match_yn,d3_all_match_yn,h1_la_match_yn,h1_mi_match_yn,h1_all_match_yn,h2_la_match_yn,h2_mi_match_yn,h2_all_match_yn,h3_la_match_yn,h3_mi_match_yn,h3_all_match_yn,c_match_yn,e_match_yn,Year,Month,Day,dayofweek,Hour,Minute,Second,dayofyear,season,Hour2,d1_match,d2_match,d3_match,h1_match,h2_match,h3_match
0,0,0,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,1,275,274,274,216,369,368,297,216,369,368,297,...,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,2020,1,17,4,12,9,36,17,winter,afternoon,TTTTT,TFFFF,TFFFF,FFF,TFF,FFF
1,1,1,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,0,114,109,56,1,175,152,56,1,175,152,56,...,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,True,False,False,True,True,2020,6,18,3,17,48,52,170,summer,afternoon,FFFFF,FFFFF,FFFFF,TTF,TFF,TFF
2,2,2,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,0,464,463,450,377,175,152,56,1,452,451,450,...,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,2020,7,8,2,20,0,10,190,summer,night,FFFFF,TTFFF,FFFFF,TFF,FFF,TFF
3,3,3,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,0,703,703,690,618,703,703,690,618,703,703,690,...,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,2020,1,13,0,18,9,34,13,winter,night,FFFFF,FFFFF,FFFFF,TFF,FFF,FFF
4,4,4,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,0,275,274,274,216,369,368,297,216,369,368,297,...,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,2020,3,9,0,20,39,22,69,spring,night,TTTTT,TFFFF,TFFFF,FFF,FFF,FFF


In [18]:
df_train = Rdata_train.copy()
df_test = Rdata_test.copy()
df_D = Rdata_D.copy()
df_H = Rdata_H.copy()
df_L = Rdata_L.copy()

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 99 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Unnamed: 0                  501951 non-null  int64 
 1   id                          501951 non-null  int64 
 2   person_attribute_a          501951 non-null  int64 
 3   person_attribute_a_under    501951 non-null  int64 
 4   person_attribute_b          501951 non-null  int64 
 5   person_prefer_c             501951 non-null  int64 
 6   person_prefer_d_1           501951 non-null  int64 
 7   person_prefer_d_2           501951 non-null  int64 
 8   person_prefer_d_3           501951 non-null  int64 
 9   person_prefer_e             501951 non-null  int64 
 10  person_prefer_f             501951 non-null  int64 
 11  person_prefer_g             501951 non-null  int64 
 12  person_prefer_h_1           501951 non-null  int64 
 13  person_prefer_h_2           5

In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 98 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  46404 non-null  int64 
 1   id                          46404 non-null  int64 
 2   person_attribute_a          46404 non-null  int64 
 3   person_attribute_a_under    46404 non-null  int64 
 4   person_attribute_b          46404 non-null  int64 
 5   person_prefer_c             46404 non-null  int64 
 6   person_prefer_d_1           46404 non-null  int64 
 7   person_prefer_d_2           46404 non-null  int64 
 8   person_prefer_d_3           46404 non-null  int64 
 9   person_prefer_e             46404 non-null  int64 
 10  person_prefer_f             46404 non-null  int64 
 11  person_prefer_g             46404 non-null  int64 
 12  person_prefer_h_1           46404 non-null  int64 
 13  person_prefer_h_2           46404 non-null  in

# 2. 범주형 encoding

In [21]:
df_train.dtypes.value_counts()

int64     65
bool      26
object     8
dtype: int64

int형에 속하는 컬럼들에 특징 값 분포
*   특징 값 분포를 통해서 Label Encoding과 Ont-Hot Encoding 방식을 결정
*   가장 많은 특징 값을 갖는 것이 'person_rn' -> 300177


*   동일한 특징 값을 갖는 특징들은 ['person_prefer_f', 'person_prefer_g', 'Y'] 라는 것을 파악 ->  이 변수들은 볼 필요가 없다.
  *   모든 값에 대해서 동일한 값을 갖는 것을 주목해야 한다.
  *   이러한 특징 값들은 추후 모델 학습에서 Drop 시키고 학습을 실시 하겠습니다.

Ont-Hot Encoding
1. 순서가 없을 때 (ex. 국가명)
2. 고유값의 개수가 많지 않으면 효율적

Label Encoding
1. 순서의 의미가 있을때 (유치원, 초등학교, 대학교 같은 등급, 사원, 대리, 과장, 부장 같은 직급?? )
2. 고유값의 개수가 많을때 (One-hot Encoding은 당연히 메모리 소비가 많으므로 효율적이진 못하다. )
3. 선형 회귀와 같은 ML알고리즘에는 적용 불가
4. 트리 계열의 ML 알고리즘은 숫자의 이러한 특성을 반영하지 않으므로 가능
5. LabelEncoder는 dataframe의 열마다 접근해야 한다.

In [22]:
df_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0)[df_train.select_dtypes('int64').apply(pd.Series.nunique, axis = 0) == 1]

person_prefer_f    1
person_prefer_g    1
Year               1
dtype: int64

In [23]:
df_train.columns

Index(['Unnamed: 0', 'id', 'person_attribute_a', 'person_attribute_a_under',
       'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1',
       'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e',
       'person_prefer_f', 'person_prefer_g', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세'

In [24]:
for column in df_train.select_dtypes(include = 'bool').columns :
    df_train[column] = df_train[column].astype(int)
    
for column in df_test.select_dtypes(include = 'bool').columns :
    df_test[column] = df_test[column].astype(int)

In [25]:
df_train2 = df_train.drop(['person_prefer_f', 'person_prefer_g', 'Year', 'Unnamed: 0', 'id', 'Minute', 'Second', 'Month'], axis = 1)
df_test2 = df_test.drop(['person_prefer_f', 'person_prefer_g', 'Year', 'Unnamed: 0', 'id', 'Minute', 'Second', 'Month'], axis = 1)
print(df_train2.shape)
print(df_test2.shape)

(501951, 91)
(46404, 90)


In [26]:
df_train2.columns

Index(['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b',
       'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2',
       'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세',
       'contents_attribute_d_소', 'contents_attribute_d_중',
    

In [27]:
df_train2 = df_train2.drop(['d1_la_match_yn', 'd1_mi_match_yn',
       'd1_sm_match_yn', 'd1_sub_match_yn', 'd1_all_match_yn',
       'd2_la_match_yn', 'd2_mi_match_yn', 'd2_sm_match_yn', 'd2_sub_match_yn',
       'd2_all_match_yn', 'd3_la_match_yn', 'd3_mi_match_yn', 'd3_sm_match_yn',
       'd3_sub_match_yn', 'd3_all_match_yn', 'h1_la_match_yn',
       'h1_mi_match_yn', 'h1_all_match_yn', 'h2_la_match_yn', 'h2_mi_match_yn',
       'h2_all_match_yn', 'h3_la_match_yn', 'h3_mi_match_yn',
       'h3_all_match_yn', 'Day', 'season', 'Hour2'], axis = 1)

df_test2 = df_test2.drop(['d1_la_match_yn', 'd1_mi_match_yn',
       'd1_sm_match_yn', 'd1_sub_match_yn', 'd1_all_match_yn',
       'd2_la_match_yn', 'd2_mi_match_yn', 'd2_sm_match_yn', 'd2_sub_match_yn',
       'd2_all_match_yn', 'd3_la_match_yn', 'd3_mi_match_yn', 'd3_sm_match_yn',
       'd3_sub_match_yn', 'd3_all_match_yn', 'h1_la_match_yn',
       'h1_mi_match_yn', 'h1_all_match_yn', 'h2_la_match_yn', 'h2_mi_match_yn',
       'h2_all_match_yn', 'h3_la_match_yn', 'h3_mi_match_yn',
       'h3_all_match_yn', 'Day', 'season', 'Hour2'], axis = 1)
print(df_train2.shape)
print(df_test2.shape)

(501951, 64)
(46404, 63)


In [28]:
df_train2.columns

Index(['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b',
       'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2',
       'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세',
       'contents_attribute_d_소', 'contents_attribute_d_중',
    

In [29]:
df_train2.dtypes.value_counts()

int64     58
object     6
dtype: int64

In [30]:
df_train2.select_dtypes(include = 'object').columns

Index(['d1_match', 'd2_match', 'd3_match', 'h1_match', 'h2_match', 'h3_match'], dtype='object')

In [31]:
df_train2.columns

Index(['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b',
       'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2',
       'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1',
       'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i',
       'contents_attribute_a', 'contents_attribute_j_under',
       'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k',
       'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m',
       'contents_attribute_e', 'contents_attribute_h', 'person_rn',
       'contents_rn', 'target', 'person_prefer_d_1_세', 'person_prefer_d_1_소',
       'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세',
       'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대',
       'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중',
       'person_prefer_d_3_대', 'contents_attribute_d_세',
       'contents_attribute_d_소', 'contents_attribute_d_중',
    

In [32]:
numerical_feats = ['dayofweek', 'Hour', 'dayofyear']

ordinal_feats = ['person_attribute_a_under', 'person_attribute_b', 'person_prefer_e', 'contents_attribute_e']

categorical_feats_oe =  ['person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
                        'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_l',
                        'contents_attribute_d', 'contents_attribute_h', 'person_rn', 'contents_rn', 
                        'person_prefer_d_1_세', 'person_prefer_d_1_소', 'person_prefer_d_1_중',
                        'person_prefer_d_1_대', 'person_prefer_d_2_세', 'person_prefer_d_2_소',
                        'person_prefer_d_2_중', 'person_prefer_d_2_대', 'person_prefer_d_3_세',
                        'person_prefer_d_3_소', 'person_prefer_d_3_중', 'person_prefer_d_3_대',
                        'contents_attribute_d_세', 'contents_attribute_d_소',
                        'contents_attribute_d_중', 'contents_attribute_d_대',
                        'person_prefer_h_1_중', 'person_prefer_h_1_대', 'person_prefer_h_2_중',
                        'person_prefer_h_2_대', 'person_prefer_h_3_중', 'person_prefer_h_3_대',
                        'contents_attribute_h_중', 'contents_attribute_h_대',
                        'contents_attribute_l_세', 'contents_attribute_l_소',
                        'contents_attribute_l_중', 'contents_attribute_l_대']

print("Number of numerical_feats : ", len(numerical_feats))
print("Number of ordinal_feats : ", len(ordinal_feats))
print("Number of categorical_feats_oe : ", len(categorical_feats_oe))

Number of numerical_feats :  3
Number of ordinal_feats :  4
Number of categorical_feats_oe :  39


In [33]:
temp = numerical_feats + ordinal_feats + categorical_feats_oe
categorical_feats_gd_ohe = df_train2.columns[~df_train2.columns.isin(temp)]
categorical_feats_gd_ohe = pd.DataFrame(categorical_feats_gd_ohe)
categorical_feats_gd_ohe = categorical_feats_gd_ohe.loc[categorical_feats_gd_ohe[0] != 'target', :]
categorical_feats_gd_ohe = categorical_feats_gd_ohe[0]
categorical_feats_gd_ohe = categorical_feats_gd_ohe.values.tolist()
print("Number of categorical_feats_gd_ohe : ", len(categorical_feats_gd_ohe))

categorical_feats_gd = ['d1_match', 'd2_match', 'd3_match', 'h1_match', 'h2_match', 'h3_match']
categorical_feats_ohe = categorical_feats_gd_ohe.copy()
for obj in categorical_feats_gd :
  categorical_feats_ohe.remove(obj)

print("Number of categorical_feats_gd : ", len(categorical_feats_gd))
print("Number of categorical_feats_ohe : ", len(categorical_feats_ohe))

Number of categorical_feats_gd_ohe :  17
Number of categorical_feats_gd :  6
Number of categorical_feats_ohe :  11


In [34]:
categorical_feats_gd

['d1_match', 'd2_match', 'd3_match', 'h1_match', 'h2_match', 'h3_match']

In [35]:
categorical_feats_ohe

['person_attribute_a',
 'person_prefer_c',
 'contents_attribute_i',
 'contents_attribute_a',
 'contents_attribute_j_under',
 'contents_attribute_j',
 'contents_attribute_c',
 'contents_attribute_k',
 'contents_attribute_m',
 'c_match_yn',
 'e_match_yn']

In [36]:
categorical_feats_oe

['person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_h',
 'person_rn',
 'contents_rn',
 'person_prefer_d_1_세',
 'person_prefer_d_1_소',
 'person_prefer_d_1_중',
 'person_prefer_d_1_대',
 'person_prefer_d_2_세',
 'person_prefer_d_2_소',
 'person_prefer_d_2_중',
 'person_prefer_d_2_대',
 'person_prefer_d_3_세',
 'person_prefer_d_3_소',
 'person_prefer_d_3_중',
 'person_prefer_d_3_대',
 'contents_attribute_d_세',
 'contents_attribute_d_소',
 'contents_attribute_d_중',
 'contents_attribute_d_대',
 'person_prefer_h_1_중',
 'person_prefer_h_1_대',
 'person_prefer_h_2_중',
 'person_prefer_h_2_대',
 'person_prefer_h_3_중',
 'person_prefer_h_3_대',
 'contents_attribute_h_중',
 'contents_attribute_h_대',
 'contents_attribute_l_세',
 'contents_attribute_l_소',
 'contents_attribute_l_중',
 'contents_attribute_l_대']

In [37]:
numerical_feats

['dayofweek', 'Hour', 'dayofyear']

In [38]:
# OrdinalEncoder

oe = OrdinalEncoder(categorical_feats_gd)
df_train2[categorical_feats_gd] = oe.fit_transform(df_train2[categorical_feats_gd], df_train2['target'])
df_test2[categorical_feats_gd] = oe.transform(df_test2[categorical_feats_gd])

df_train2[categorical_feats_gd] = df_train2[categorical_feats_gd].astype('int64')
df_test2[categorical_feats_gd] = df_test2[categorical_feats_gd].astype('int64')

In [39]:
df_train2.head()

,person_attribute_a,person_attribute_a_under,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_under,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,target,person_prefer_d_1_세,person_prefer_d_1_소,person_prefer_d_1_중,person_prefer_d_1_대,person_prefer_d_2_세,person_prefer_d_2_소,person_prefer_d_2_중,person_prefer_d_2_대,person_prefer_d_3_세,person_prefer_d_3_소,person_prefer_d_3_중,person_prefer_d_3_대,contents_attribute_d_세,contents_attribute_d_소,contents_attribute_d_중,contents_attribute_d_대,person_prefer_h_1_중,person_prefer_h_1_대,person_prefer_h_2_중,person_prefer_h_2_대,person_prefer_h_3_중,person_prefer_h_3_대,contents_attribute_h_중,contents_attribute_h_대,contents_attribute_l_세,contents_attribute_l_소,contents_attribute_l_중,contents_attribute_l_대,c_match_yn,e_match_yn,dayofweek,Hour,dayofyear,d1_match,d2_match,d3_match,h1_match,h2_match,h3_match
0,1,4,3,5,275,370,369,8,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,1,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,316,3,398,94,368,58,422,94,1607,1606,1605,2016,0,0,4,12,17,1,1,1,1,1,1
1,1,3,4,1,114,181,175,4,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,0,114,109,56,1,175,152,56,1,175,152,56,1,275,274,274,216,417,94,400,94,399,94,417,94,1607,1606,1605,2016,1,1,3,17,170,2,2,2,2,1,2
2,2,0,3,5,464,175,452,3,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,0,464,463,450,377,175,152,56,1,452,451,450,377,92,91,56,1,364,48,528,250,366,48,363,48,1599,1595,1572,2016,0,0,2,20,190,2,3,2,3,2,2
3,2,0,2,5,703,705,704,3,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,0,703,703,690,618,703,703,690,618,703,703,690,618,275,274,274,216,379,71,495,226,315,1,381,71,1607,1606,1605,2016,0,1,0,18,13,2,2,2,3,2,1
4,1,3,4,5,275,370,369,4,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,0,275,274,274,216,369,368,297,216,369,368,297,216,275,274,274,216,483,208,481,208,480,208,381,71,1607,1606,1605,2016,0,1,0,20,69,1,1,1,1,2,1


In [40]:
print(df_train2.dtypes.value_counts())
print(df_test2.dtypes.value_counts())

int64    64
dtype: int64
int64    63
dtype: int64


In [41]:
###################################################################################################################
# 여기서부터

# a = pd.get_dummies(df_train2[categorical_feats_gd])
# df_train3 = pd.concat([df_train2, a], axis = 1)
# for obj in categorical_feats_gd :
#     df_train3 = df_train3.drop(obj, axis = 1)
    
# print(df_train3.shape)

# b = pd.get_dummies(df_test2[categorical_feats_gd])
# df_test3 = pd.concat([df_test2, b], axis = 1)
# for obj in categorical_feats_gd :
#     df_test3 = df_test3.drop(obj, axis = 1)

# df_test3['season_fall'] = 0
# df_test3['season_spring'] = 0
# df_test3['season_summer'] = 0

# print(df_test3.shape)
###################################################################################################################

In [42]:
# df_train3.dtypes.value_counts()

In [43]:
###################################################################################################################
# One Hot Enconding

# ohe = OneHotEncoder()

# c = ohe.fit_transform(df_train3[categorical_feats_ohe]).toarray()
# d = ohe.transform(df_test3[categorical_feats_ohe]).toarray()

# ohe_columns = ohe.get_feature_names(categorical_feats_ohe)
# cc = pd.DataFrame(c, columns = ohe_columns)
# dd = pd.DataFrame(d, columns = ohe_columns)
# print(cc.shape)
# print(dd.shape)


# df_train4 = pd.concat([df_train3, cc], axis = 1)
# for obj in categorical_feats_ohe :
#     df_train4 = df_train4.drop(obj, axis = 1)

# print(len(categorical_feats_ohe))
# print(df_train3.shape)
# print(df_train4.shape)

# df_test4 = pd.concat([df_test3, dd], axis = 1)
# for obj in categorical_feats_ohe :
 #    df_test4 = df_test4.drop(obj, axis = 1)

# print(len(categorical_feats_ohe))
# print(df_test3.shape)
# print(df_test4.shape)
###################################################################################################################

In [44]:
###################################################################################################################
# 숫자 -> 문자

# df_train4[categorical_feats_oe] = df_train4[categorical_feats_oe].astype(str)
# print(df_train4.dtypes)

# df_test4[categorical_feats_oe] = df_test4[categorical_feats_oe].astype(str)
# print(df_test4.dtypes)
###################################################################################################################

In [45]:
###################################################################################################################
# OrdinalEncoder

# oe = OrdinalEncoder(categorical_feats_oe)
# df_train4[categorical_feats_oe] = oe.fit_transform(df_train4[categorical_feats_oe], df_train4['target'])
# df_test4[categorical_feats_oe] = oe.transform(df_test4[categorical_feats_oe])

# df_train4['person_rn'] = df_train4['person_rn'].astype('int64')
# df_train4['contents_rn'] = df_train4['contents_rn'].astype('int64')
# df_test4['person_rn'] = df_test4['person_rn'].astype('int64')
# df_test4['contents_rn'] = df_test4['contents_rn'].astype('int64')
###################################################################################################################

In [46]:
df_train3 = df_train2.copy()
df_test3 = df_test2.copy()

# 3. 클러스터링 및 데이터 분할

In [47]:
# 클러스터링 구성
# 타겟을 결정짓는 뚜렷한 특징을 갖는 피쳐를 찾지 못해 clustering 시도

# kmeans_train = df_train3.drop(['target'], axis = 1)
# kmeans = KMeans(n_clusters = 36, init = 'k-means++', n_init = 10, max_iter = 300, tol = 1e-04, random_state = 39).fit(kmeans_train)
# df_train3['cluster'] = kmeans.predict(kmeans_train)
# df_test3['cluster'] = kmeans.predict(df_test3)

In [48]:
# df_train3['cluster'] = df_train3['cluster'].astype('int64')
# df_test3['cluster'] = df_test3['cluster'].astype('int64')

# df_train3.dtypes.value_counts()

In [49]:
df_train4 = df_train3.copy()
df_test4 = df_test3.copy()

In [50]:
print(df_train4.shape)
print(df_test4.shape)

(501951, 64)
(46404, 63)


In [51]:
# StandardScale
# 컬럼 정규화

temp_columns2 = numerical_feats
scaler = StandardScaler()
df_train4[temp_columns2] = scaler.fit_transform(df_train4[temp_columns2])
df_test4[temp_columns2] = scaler.transform(df_test4[temp_columns2])

In [52]:
print(df_train4.dtypes.value_counts())
print(df_test4.dtypes.value_counts())

int64      61
float64     3
dtype: int64
int64      60
float64     3
dtype: int64


In [53]:
numerical_feats

['dayofweek', 'Hour', 'dayofyear']

In [54]:
# sorted(df_train4['Month'].unique())

In [55]:
# df_train6_1 = df_train4.loc[df_train4['Month'] < 0.7, :]
# df_train6_1.reset_index(drop = True, inplace = True)
# X_part_train = df_train6_1.drop('target', axis = 1)
# y_part_train = df_train6_1['target']

# df_train6_2 = df_train4.loc[(df_train4['Month'] > 0.7) & (df_train4['Month'] < 1.5)]
# df_train6_2.reset_index(drop = True, inplace = True)
# X_val = df_train6_2.drop('target', axis = 1)
# y_val = df_train6_2['target']

# df_train6_3 = df_train4.loc[df_train4['Month']> 1.5]
# df_train6_3.reset_index(drop = True, inplace = True)
# X_part_test = df_train6_3.drop('target', axis = 1)
# y_part_test = df_train6_3['target']

# df_train6_4 = df_train4.loc[df_train4['Month'] < 1.5]
# df_train6_4.reset_index(drop = True, inplace = True)
# X_train = df_train6_4.drop('target', axis = 1)
# y_train = df_train6_4['target']

# X_test = df_test4.copy()

# print(X_part_train.shape)
# print(y_part_train.shape)
# print(X_val.shape)
# print(y_val.shape)
# print(X_part_test.shape)
# print(y_part_test.shape)

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(df_train4.shape)

In [56]:
X_train = df_train4.drop('target', axis = 1)
y_train = df_train4['target']
X_test = df_test4.copy()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(501951, 63)
(501951,)
(46404, 63)


In [57]:
tt1 = X_train.columns.tolist() 
tt2 = numerical_feats
print(len(tt1))
print(len(tt2))
tt3 = []
for i in tt1 :
  if i not in tt2 :
    tt3.append(i)
print(len(tt3))
print(tt3)

cat_cols = tt3

63
3
60
['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i', 'contents_attribute_a', 'contents_attribute_j_under', 'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e', 'contents_attribute_h', 'person_rn', 'contents_rn', 'person_prefer_d_1_세', 'person_prefer_d_1_소', 'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세', 'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대', 'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중', 'person_prefer_d_3_대', 'contents_attribute_d_세', 'contents_attribute_d_소', 'contents_attribute_d_중', 'contents_attribute_d_대', 'person_prefer_h_1_중', 'person_prefer_h_1_대', 'person_prefer_h_2_중', 'person_

In [58]:
temp = X_train.columns[X_train.nunique() > 2].tolist()
temp2 = set(temp)
cat_cols2 = set(cat_cols)
cat_cols3 = list(temp2 & cat_cols2)
print(len(cat_cols3))

55


# 4. 1차 모델 생성 (randomforest, xgboost, lightgbm, extratree, catboost, tabnet, hgbm)

### 1) randomforest

In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
 #       'min_samples_split' : trial.suggest_int('min_samples_split', 1, 150),
 #       'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 60),
        "n_estimators":trial.suggest_int("n_estimators", 100, 1000),
        "max_depth":trial.suggest_int("max_depth", 3, 30)
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

    model = RandomForestClassifier(**params_cat)
    model.fit(X_part_train, y_part_train)
    rf_pred = model.predict(X_val)
    f1 = f1_score(y_val, rf_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "rf_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 20)

[I 2022-01-26 04:40:04,320] A new study created in memory with name: rf_parameter_opt
[I 2022-01-26 04:55:58,353] Trial 0 finished with value: 0.6457860176940694 and parameters: {'n_estimators': 592, 'max_depth': 25}. Best is trial 0 with value: 0.6457860176940694.
[I 2022-01-26 05:02:29,797] Trial 1 finished with value: 0.6060471492286602 and parameters: {'n_estimators': 839, 'max_depth': 6}. Best is trial 0 with value: 0.6457860176940694.
[I 2022-01-26 05:16:18,645] Trial 2 finished with value: 0.6403135983717008 and parameters: {'n_estimators': 642, 'max_depth': 17}. Best is trial 0 with value: 0.6457860176940694.
[I 2022-01-26 05:26:37,371] Trial 3 finished with value: 0.6385378294134316 and parameters: {'n_estimators': 517, 'max_depth': 16}. Best is trial 0 with value: 0.6457860176940694.
[I 2022-01-26 05:45:15,414] Trial 4 finished with value: 0.642474524989772 and parameters: {'n_estimators': 670, 'max_depth': 28}. Best is trial 0 with value: 0.6457860176940694.
[I 2022-01-26 06

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6457860176940694
Best trial : {'n_estimators': 592, 'max_depth': 25}


In [ ]:
# 시각화
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

rf_val = np.zeros((X_train.shape[0], 2))
rf_test = np.zeros((X_test.shape[0], 2))

print(rf_val.shape)
print(rf_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_rf = RandomForestClassifier(
                        random_state = 39,
 #                       min_samples_split = study.best_trial.params['min_samples_split'],
 #                       min_samples_leaf = study.best_trial.params['min_samples_leaf'],
                        n_estimators = study.best_trial.params['n_estimators'],
                        max_depth = study.best_trial.params['max_depth'])

    optuna_rf.fit(X_train.loc[i_trn, :], y_train[i_trn])

    rf_val[i_val, :] = optuna_rf.predict_proba(X_train.loc[i_val, :])
    rf_test += optuna_rf.predict_proba(X_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [ ]:
print(rf_val.shape)
print(rf_test.shape)

rf_pred = rf_val[:, 1]
rf_pred2 = np.where(rf_pred >= 0.5, 1, 0)
print(f1_score(y_train, rf_pred2))

(501951, 2)
(46404, 2)
0.643998194344183


In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = rf_pred
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6904926413143122


In [ ]:
# 저장

rf_val_file = pd.DataFrame(rf_val, columns = ['0', '1'])
# rf_part_test_file = pd.DataFrame(rf_part_test, columns = ['0', '1'])
rf_test_file = pd.DataFrame(rf_test, columns = ['0', '1'])
rf_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/rf_val.csv', index = False)
# rf_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/rf_part_test.csv', index = False)
rf_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/rf_test.csv', index = False)

### 2) lightgbm

In [ ]:
def objective(trial : Trial) -> float :

    params_cat = {
        'random_state' : 39,
        "learning_rate" : 0.05,
                        'n_estimators' : trial.suggest_int('n_estimators', 100, 1000), 
                        'reg_alpha' : trial.suggest_float("reg_alpha", 1e-8, 10), 
 #                       'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 10), 
                        'max_depth' : trial.suggest_int("max_depth", 3, 20), 
                        'num_leaves' : trial.suggest_int("num_leaves", 2, 150), 
 #                       'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.4, 1.0), 
                        'subsample' : trial.suggest_float("subsample", 0.3, 1.0), 
 #                       'subsample_freq' : trial.suggest_int("subsample_freq", 1, 20), 
                        'min_child_samples' : trial.suggest_int("min_child_samples", 5, 100), 
 #                       'max_bin' : trial.suggest_int("max_bin", 200, 500),
                        "verbosity" : -1
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

    model = LGBMClassifier(**params_cat)
    model.fit(X_part_train, y_part_train, eval_set = [(X_part_train, y_part_train), (X_val, y_val)],
              early_stopping_rounds = 100, verbose = False)

    lgbm_pred = model.predict(X_val)
    f1 = f1_score(y_val, lgbm_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "lgbm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-01-26 10:15:12,187] A new study created in memory with name: lgbm_parameter_opt
[I 2022-01-26 10:15:51,965] Trial 0 finished with value: 0.6395254070241276 and parameters: {'n_estimators': 592, 'reg_alpha': 7.9789902148470615, 'max_depth': 17, 'num_leaves': 20, 'subsample': 0.7214014047540065, 'min_child_samples': 55}. Best is trial 0 with value: 0.6395254070241276.
[I 2022-01-26 10:16:42,899] Trial 1 finished with value: 0.6486713260042525 and parameters: {'n_estimators': 517, 'reg_alpha': 4.7144574495307, 'max_depth': 14, 'num_leaves': 139, 'subsample': 0.8708520410253093, 'min_child_samples': 95}. Best is trial 1 with value: 0.6486713260042525.
[I 2022-01-26 10:17:59,798] Trial 2 finished with value: 0.6496320327677453 and parameters: {'n_estimators': 928, 'reg_alpha': 4.148307969361902, 'max_depth': 17, 'num_leaves': 142, 'subsample': 0.7396322372647702, 'min_child_samples': 29}. Best is trial 2 with value: 0.6496320327677453.
[I 2022-01-26 10:18:32,333] Trial 3 finished wi

In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6522588803310725
Best trial : {'n_estimators': 944, 'reg_alpha': 8.05777417080913, 'max_depth': 16, 'num_leaves': 149, 'subsample': 0.4801405586816589, 'min_child_samples': 10}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

lgbm_val = np.zeros((X_train.shape[0], 2))
lgbm_test = np.zeros((X_test.shape[0], 2))

print(lgbm_val.shape)
print(lgbm_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_lgbm = LGBMClassifier(
                        random_state = 39,
                        n_estimators = study.best_trial.params['n_estimators'], 
                        reg_alpha = study.best_trial.params['reg_alpha'], 
  #                      reg_lambda = study.best_trial.params['reg_lambda'], 
                        max_depth = study.best_trial.params['max_depth'], 
                        num_leaves = study.best_trial.params['num_leaves'], 
  #                      colsample_bytree = study.best_trial.params['colsample_bytree'], 
                        subsample = study.best_trial.params['subsample'], 
   #                     subsample_freq = study.best_trial.params['subsample_freq'], 
                        min_child_samples = study.best_trial.params['min_child_samples'], 
   #                     max_bin = study.best_trial.params['max_bin'],
                        verbose = 100)

    optuna_lgbm.fit(X_train.loc[i_trn, :], y_train[i_trn], eval_set = [(X_train.loc[i_val, :], y_train[i_val])], early_stopping_rounds = 100, eval_metric = 'auc')

    lgbm_val[i_val, :] = optuna_lgbm.predict_proba(X_train.loc[i_val, :])
    lgbm_test += optuna_lgbm.predict_proba(X_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
[1]	valid_0's auc: 0.636609	valid_0's binary_logloss: 0.687286
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.640861	valid_0's binary_logloss: 0.682413
[3]	valid_0's auc: 0.642128	valid_0's binary_logloss: 0.678422
[4]	valid_0's auc: 0.643677	valid_0's binary_logloss: 0.675099
[5]	valid_0's auc: 0.645509	valid_0's binary_logloss: 0.672196
[6]	valid_0's auc: 0.646663	valid_0's binary_logloss: 0.669721
[7]	valid_0's auc: 0.647584	valid_0's binary_logloss: 0.667552
[8]	valid_0's auc: 0.648397	valid_0's binary_logloss: 0.665775
[9]	valid_0's auc: 0.649093	valid_0's binary_logloss: 0.664147
[10]	valid_0's auc: 0.649667	valid_0's binary_logloss: 0.662823
[11]	valid_0's auc: 0.650214	valid_0's binary_logloss: 0.66166
[12]	valid_0's auc: 0.650957	valid_0's binary_logloss: 0.660516
[13]	valid_0's auc: 0.652116	valid_0's binary_logloss: 0.659437
[14]	valid_0's auc: 0.652776	valid_0's binary_logloss: 0.658567
[15

In [ ]:
print(lgbm_val.shape)
print(lgbm_test.shape)

lgbm_pred = lgbm_val[:, 1]
lgbm_pred2 = np.where(lgbm_pred >= 0.5, 1, 0)
print(f1_score(y_train, lgbm_pred2))

(501951, 2)
(46404, 2)
0.6474294296607233


In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = lgbm_pred
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6960970959003617


In [ ]:
# 저장

lgbm_val_file = pd.DataFrame(lgbm_val, columns = ['0', '1'])
# Vlgbm_part_test_file = pd.DataFrame(lgbm_part_test, columns = ['0', '1'])
lgbm_test_file = pd.DataFrame(lgbm_test, columns = ['0', '1'])
lgbm_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/lgbm_val.csv', index = False)
# lgbm_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/lgbm_part_test.csv', index = False)
lgbm_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/lgbm_test.csv', index = False)

### 3) xgboost

In [ ]:
def objective(trial : Trial) -> float :

    params_xgb = {
        "random_state" : 39,
        'max_depth' : trial.suggest_int("max_depth", 1, 16),
        'colsample_bytree' : trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'colsample_bylevel' : trial.suggest_float("colsample_bylevel", 0.1, 1.0),
   #     'colsample_bynode' : trial.suggest_float("colsample_bynode", 0.1, 1.0),
        'min_child_weight' : trial.suggest_float("min_child_weight", 0.1, 300.0),
        'gamma' : trial.suggest_loguniform('gamma', 1e-3, 10.0),
        'subsample' : trial.suggest_float("subsample", 0.1, 1.0),
  #      'reg_alpha' : trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
  #      'reg_lambda' : trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 1, 16)
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

    model = xgb.XGBClassifier(**params_xgb)
    model.fit(X_part_train, y_part_train, eval_set = [(X_val, y_val)], early_stopping_rounds = 100, eval_metric = 'auc')

    xgb_pred = model.predict(X_val)
    f1 = f1_score(y_val, xgb_pred)
    
    return f1

In [ ]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "xgb_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 20)

[I 2022-01-26 12:22:31,556] A new study created in memory with name: xgb_parameter_opt


[0]	validation_0-auc:0.624276
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.634587
[2]	validation_0-auc:0.636594
[3]	validation_0-auc:0.641569
[4]	validation_0-auc:0.644675
[5]	validation_0-auc:0.645515
[6]	validation_0-auc:0.646321
[7]	validation_0-auc:0.646891
[8]	validation_0-auc:0.647844
[9]	validation_0-auc:0.648592
[10]	validation_0-auc:0.649268
[11]	validation_0-auc:0.649967
[12]	validation_0-auc:0.651
[13]	validation_0-auc:0.651732
[14]	validation_0-auc:0.652369
[15]	validation_0-auc:0.652886
[16]	validation_0-auc:0.653363
[17]	validation_0-auc:0.653537
[18]	validation_0-auc:0.654031
[19]	validation_0-auc:0.654588
[20]	validation_0-auc:0.655401
[21]	validation_0-auc:0.655757
[22]	validation_0-auc:0.656309
[23]	validation_0-auc:0.657134
[24]	validation_0-auc:0.657645
[25]	validation_0-auc:0.658293
[26]	validation_0-auc:0.658492
[27]	validation_0-auc:0.659033
[28]	validation_0-auc:0.659235
[29]	validation_0-auc:0.65963
[30]	validation_0-a

[I 2022-01-26 12:40:01,208] Trial 0 finished with value: 0.6419757846841982 and parameters: {'max_depth': 9, 'colsample_bytree': 0.8181091191543447, 'colsample_bylevel': 0.8383616919085431, 'min_child_weight': 36.70275506237732, 'gamma': 0.2558633178279746, 'subsample': 0.5729631250281253, 'n_estimators': 517}. Best is trial 0 with value: 0.6419757846841982.


[0]	validation_0-auc:0.621265
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.631934
[2]	validation_0-auc:0.634618
[3]	validation_0-auc:0.637182
[4]	validation_0-auc:0.640124
[5]	validation_0-auc:0.641242
[6]	validation_0-auc:0.641712
[7]	validation_0-auc:0.642027
[8]	validation_0-auc:0.64252
[9]	validation_0-auc:0.643813
[10]	validation_0-auc:0.644481
[11]	validation_0-auc:0.644635
[12]	validation_0-auc:0.645632
[13]	validation_0-auc:0.646187
[14]	validation_0-auc:0.64742
[15]	validation_0-auc:0.647697
[16]	validation_0-auc:0.648373
[17]	validation_0-auc:0.64917
[18]	validation_0-auc:0.649718
[19]	validation_0-auc:0.650227
[20]	validation_0-auc:0.650826
[21]	validation_0-auc:0.651394
[22]	validation_0-auc:0.65195
[23]	validation_0-auc:0.652542
[24]	validation_0-auc:0.653299
[25]	validation_0-auc:0.653696
[26]	validation_0-auc:0.654074
[27]	validation_0-auc:0.654756
[28]	validation_0-auc:0.655041
[29]	validation_0-auc:0.655283
[30]	validation_0-a

[I 2022-01-26 12:51:10,533] Trial 1 finished with value: 0.6466618945359103 and parameters: {'max_depth': 8, 'colsample_bytree': 0.6694415585895983, 'colsample_bylevel': 0.9330974924253972, 'min_child_weight': 244.6693244335575, 'gamma': 5.9946938641166785, 'subsample': 0.9276266278535859, 'n_estimators': 473}. Best is trial 1 with value: 0.6466618945359103.


...

[I 2022-01-26 14:48:07,036] Trial 17 finished with value: 0.6353954489902811 and parameters: {'max_depth': 13, 'colsample_bytree': 0.398291915358281, 'colsample_bylevel': 0.43712474931698897, 'min_child_weight': 114.16043163551586, 'gamma': 0.003820255741291654, 'subsample': 0.26806574490519497, 'n_estimators': 991}. Best is trial 4 with value: 0.6476642762080752.


[0]	validation_0-auc:0.615838
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.624656
[2]	validation_0-auc:0.62911
[3]	validation_0-auc:0.632403
[4]	validation_0-auc:0.635306
[5]	validation_0-auc:0.636076
[6]	validation_0-auc:0.638825
[7]	validation_0-auc:0.640259
[8]	validation_0-auc:0.641837
[9]	validation_0-auc:0.643987
[10]	validation_0-auc:0.644912
[11]	validation_0-auc:0.645493
[12]	validation_0-auc:0.646401
[13]	validation_0-auc:0.646457
[14]	validation_0-auc:0.647582
[15]	validation_0-auc:0.647855
[16]	validation_0-auc:0.648203
[17]	validation_0-auc:0.649289
[18]	validation_0-auc:0.649709
[19]	validation_0-auc:0.650258
[20]	validation_0-auc:0.650642
[21]	validation_0-auc:0.651525
[22]	validation_0-auc:0.651873
[23]	validation_0-auc:0.652391
[24]	validation_0-auc:0.652989
[25]	validation_0-auc:0.653462
[26]	validation_0-auc:0.654146
[27]	validation_0-auc:0.654936
[28]	validation_0-auc:0.655085
[29]	validation_0-auc:0.656214
[30]	validation_

[I 2022-01-26 14:58:58,512] Trial 18 finished with value: 0.6460792926026392 and parameters: {'max_depth': 16, 'colsample_bytree': 0.22617218440030906, 'colsample_bylevel': 0.6171091682338747, 'min_child_weight': 200.74199234482037, 'gamma': 0.043300940670538315, 'subsample': 0.8346440270267889, 'n_estimators': 585}. Best is trial 4 with value: 0.6476642762080752.


[0]	validation_0-auc:0.602135
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.621997
[2]	validation_0-auc:0.623847
[3]	validation_0-auc:0.626901
[4]	validation_0-auc:0.630141
[5]	validation_0-auc:0.630566
[6]	validation_0-auc:0.632108
[7]	validation_0-auc:0.631659
[8]	validation_0-auc:0.631916
[9]	validation_0-auc:0.633042
[10]	validation_0-auc:0.632606
[11]	validation_0-auc:0.633598
[12]	validation_0-auc:0.634363
[13]	validation_0-auc:0.634588
[14]	validation_0-auc:0.635225
[15]	validation_0-auc:0.636103
[16]	validation_0-auc:0.637071
[17]	validation_0-auc:0.63797
[18]	validation_0-auc:0.6385
[19]	validation_0-auc:0.639227
[20]	validation_0-auc:0.639791
[21]	validation_0-auc:0.640468
[22]	validation_0-auc:0.640587
[23]	validation_0-auc:0.641395
[24]	validation_0-auc:0.6419
[25]	validation_0-auc:0.642306
[26]	validation_0-auc:0.643134
[27]	validation_0-auc:0.643596
[28]	validation_0-auc:0.643792
[29]	validation_0-auc:0.6448
[30]	validation_0-auc:

[I 2022-01-26 15:03:43,791] Trial 19 finished with value: 0.6423521082291258 and parameters: {'max_depth': 6, 'colsample_bytree': 0.3411652175154303, 'colsample_bylevel': 0.7655537681682352, 'min_child_weight': 261.5072528242606, 'gamma': 0.0027592162146457496, 'subsample': 0.7156893316929, 'n_estimators': 425}. Best is trial 4 with value: 0.6476642762080752.


In [ ]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6476642762080752
Best trial : {'max_depth': 9, 'colsample_bytree': 0.2688375158934132, 'colsample_bylevel': 0.7034023041696267, 'min_child_weight': 164.66987696898278, 'gamma': 0.00422380262928549, 'subsample': 0.9392466147658407, 'n_estimators': 589}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

xgb_val = np.zeros((X_train.shape[0], 2))
xgb_test = np.zeros((X_test.shape[0], 2))

print(xgb_val.shape)
print(xgb_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_xgb = xgb.XGBClassifier(
                        random_state = 39,
                        max_depth = study.best_trial.params['max_depth'], 
                        colsample_bytree = study.best_trial.params['colsample_bytree'], 
                        colsample_bylevel = study.best_trial.params['colsample_bylevel'], 
    #                    colsample_bynode = study.best_trial.params['colsample_bynode'], 
                        min_child_weight = study.best_trial.params['min_child_weight'], 
                        gamma = study.best_trial.params['gamma'], 
                        subsample = study.best_trial.params['subsample'], 
     #                   reg_alpha = study.best_trial.params['reg_alpha'], 
     #                   reg_lambda = study.best_trial.params['reg_lambda'], 
                        n_estimators = study.best_trial.params['n_estimators'],
                        verbose = False)

    optuna_xgb.fit(X_train.loc[i_trn, :], y_train[i_trn], eval_set = [(X_train.loc[i_val, :], y_train[i_val])], early_stopping_rounds = 50, eval_metric = 'auc')

    xgb_val[i_val, :] = optuna_xgb.predict_proba(X_train.loc[i_val, :])
    xgb_test += optuna_xgb.predict_proba(X_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
[0]	validation_0-auc:0.615157
Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.631408
[2]	validation_0-auc:0.631765
[3]	validation_0-auc:0.635749
[4]	validation_0-auc:0.636259
[5]	validation_0-auc:0.637098
[6]	validation_0-auc:0.636952
[7]	validation_0-auc:0.637355
[8]	validation_0-auc:0.638756
[9]	validation_0-auc:0.640785
[10]	validation_0-auc:0.642
[11]	validation_0-auc:0.643265
[12]	validation_0-auc:0.644389
[13]	validation_0-auc:0.644607
[14]	validation_0-auc:0.645522
[15]	validation_0-auc:0.645752
[16]	validation_0-auc:0.646292
[17]	validation_0-auc:0.647483
[18]	validation_0-auc:0.648134
[19]	validation_0-auc:0.648803
[20]	validation_0-auc:0.649427
[21]	validation_0-auc:0.650188
[22]	validation_0-auc:0.650584
[23]	validation_0-auc:0.651276
[24]	validation_0-auc:0.651504
[25]	validation_0-auc:0.652011
[26]	validation_0-auc:0.652723
[27]	validation_0-auc:0.653425
[28]	validation_0-auc:0.653598
[2

In [ ]:
print(xgb_val.shape)
print(xgb_test.shape)

xgb_pred = xgb_val[:, 1]
xgb_pred2 = np.where(xgb_pred >= 0.5, 1, 0)
print(f1_score(y_train, xgb_pred2))

(501951, 2)
(46404, 2)
0.6471148739716519


In [ ]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = xgb_pred
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6950282795110381


In [ ]:
# 저장

xgb_val_file = pd.DataFrame(xgb_val, columns = ['0', '1'])
# xgb_part_test_file = pd.DataFrame(xgb_part_test, columns = ['0', '1'])
xgb_test_file = pd.DataFrame(xgb_test, columns = ['0', '1'])
xgb_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/xgb_val.csv', index = False)
# xgb_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/xgb_part_test.csv', index = False)
xgb_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/xgb_test.csv', index = False)

### 4) extratree

In [73]:
def objective(trial : Trial) -> float :

    params_etc = {
        "random_state" : 39,
        'n_jobs' : -1,
        'max_depth' : trial.suggest_int("max_depth", 1, 20),
        'n_estimators' : trial.suggest_int("n_estimators", 100, 1000),
 #       'min_samples_split' : trial.suggest_int('min_samples_split', 2, 15),
 #       'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 15),
 #       'max_features' : trial.suggest_uniform('max_features', 0.1, 1.0)
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

    model = ExtraTreesClassifier(**params_etc)
    model.fit(X_part_train, y_part_train)

    etc_pred = model.predict(X_val)
    f1 = f1_score(y_val, etc_pred)
    
    return f1

In [74]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "etc_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 20)

[I 2022-01-27 07:19:54,152] A new study created in memory with name: etc_parameter_opt
[I 2022-01-27 07:22:40,296] Trial 0 finished with value: 0.6146247140245246 and parameters: {'max_depth': 11, 'n_estimators': 818}. Best is trial 0 with value: 0.6146247140245246.
[I 2022-01-27 07:23:44,835] Trial 1 finished with value: 0.6315968193780243 and parameters: {'max_depth': 17, 'n_estimators': 209}. Best is trial 1 with value: 0.6315968193780243.
[I 2022-01-27 07:26:02,690] Trial 2 finished with value: 0.6255585005679667 and parameters: {'max_depth': 13, 'n_estimators': 573}. Best is trial 1 with value: 0.6315968193780243.
[I 2022-01-27 07:27:40,503] Trial 3 finished with value: 0.612597761079589 and parameters: {'max_depth': 10, 'n_estimators': 524}. Best is trial 1 with value: 0.6315968193780243.
[I 2022-01-27 07:31:25,755] Trial 4 finished with value: 0.6203685225836493 and parameters: {'max_depth': 13, 'n_estimators': 934}. Best is trial 1 with value: 0.6315968193780243.
[I 2022-01-27 

In [75]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6376100203114421
Best trial : {'max_depth': 17, 'n_estimators': 950}


In [76]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [77]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [78]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [80]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

etc_val = np.zeros((X_train.shape[0], 2))
etc_test = np.zeros((X_test.shape[0], 2))

print(etc_val.shape)
print(etc_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_etc = ExtraTreesClassifier(
                        random_state = 39,
   #                     min_samples_split = trial.suggest_int("min_samples_split", 1, 20), 
   #                     min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20), 
   #                     max_features = trial.suggest_int("max_features", 1, 20),
                        max_depth = study.best_trial.params['max_depth'],
                        n_estimators = study.best_trial.params['n_estimators'],
                        n_jobs = -1)

    optuna_etc.fit(X_train.loc[i_trn, :], y_train[i_trn])

    etc_val[i_val, :] = optuna_etc.predict_proba(X_train.loc[i_val, :])
    etc_test += optuna_etc.predict_proba(X_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [81]:
print(etc_val.shape)
print(etc_test.shape)

etc_pred = etc_val[:, 1]
etc_pred2 = np.where(etc_pred >= 0.5, 1, 0)
print(f1_score(y_train, etc_pred2))


(501951, 2)
(46404, 2)
0.6339545285830832


In [82]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = etc_pred
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6853655297419335


In [83]:
# 저장

etc_val_file = pd.DataFrame(etc_val, columns = ['0', '1'])
# etc_part_test_file = pd.DataFrame(etc_part_test, columns = ['0', '1'])
etc_test_file = pd.DataFrame(etc_test, columns = ['0', '1'])
etc_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/etc_val.csv', index = False)
# etc_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/etc_part_test.csv', index = False)
etc_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/etc_test.csv', index = False)

### 5) catboost

In [49]:
print(len(tt3))
print(tt3)

cat_cols = tt3

60
['person_attribute_a', 'person_attribute_a_under', 'person_attribute_b', 'person_prefer_c', 'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3', 'person_prefer_e', 'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3', 'contents_attribute_i', 'contents_attribute_a', 'contents_attribute_j_under', 'contents_attribute_j', 'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l', 'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e', 'contents_attribute_h', 'person_rn', 'contents_rn', 'person_prefer_d_1_세', 'person_prefer_d_1_소', 'person_prefer_d_1_중', 'person_prefer_d_1_대', 'person_prefer_d_2_세', 'person_prefer_d_2_소', 'person_prefer_d_2_중', 'person_prefer_d_2_대', 'person_prefer_d_3_세', 'person_prefer_d_3_소', 'person_prefer_d_3_중', 'person_prefer_d_3_대', 'contents_attribute_d_세', 'contents_attribute_d_소', 'contents_attribute_d_중', 'contents_attribute_d_대', 'person_prefer_h_1_중', 'person_prefer_h_1_대', 'person_prefer_h_2_중', 'person_prefe

In [50]:
temp = X_train.columns[X_train.nunique() > 2].tolist()
temp2 = set(temp)
cat_cols2 = set(cat_cols)
cat_cols3 = list(temp2 & cat_cols2)
print(len(cat_cols3))

55


In [51]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 5, 30.00),
        "n_estimators":trial.suggest_int("n_estimators", 100, 1000),
        "max_depth":trial.suggest_int("max_depth", 8, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': 'Iter'
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

    model = CatBoostClassifier(**params_cat,  task_type = "GPU")
    model.fit(X_part_train, y_part_train, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = cat_cols3, verbose = False)

    cat_pred = model.predict(X_val)
    f1 = f1_score(y_val, cat_pred)
    
    return f1

In [52]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 50)

[I 2022-01-26 18:36:38,354] A new study created in memory with name: cat_parameter_opt
[I 2022-01-26 18:39:42,156] Trial 0 finished with value: 0.6321131118007149 and parameters: {'learning_rate': 0.012410186619312995, 'bagging_temperature': 20.886040962658566, 'n_estimators': 839, 'max_depth': 9, 'random_strength': 60, 'max_bin': 358, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.6321131118007149.
[I 2022-01-26 19:12:48,456] Trial 1 finished with value: 0.6318671882453964 and parameters: {'learning_rate': 0.008767817244598053, 'bagging_temperature': 15.535180162253402, 'n_estimators': 934, 'max_depth': 15, 'random_strength': 95, 'max_bin': 476, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.6321131118007149.
[I 2022-01-26 19:15:45,936] Trial 2 finished with value: 0.6309891475833054 and parameters: {'learning_rate': 0.04483569690392285, 'bagging_temperature': 27.246046846093954, 'n_estimators': 665, 'max_depth': 10, 'random_strength': 59, 'max_bin': 458, 'one_hot_max_si

In [53]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.677509563135399
Best trial : {'learning_rate': 0.06213018921981329, 'bagging_temperature': 5.393442135283861, 'n_estimators': 167, 'max_depth': 8, 'random_strength': 48, 'max_bin': 465, 'one_hot_max_size': 4}


In [54]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [55]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [56]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [57]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train.shape[0], 2))
cat_test = np.zeros((X_test.shape[0], 2))

print(cat_val.shape)
print(cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = study.best_trial.params['learning_rate'],
        bagging_temperature = study.best_trial.params['bagging_temperature'],
        n_estimators = study.best_trial.params['n_estimators'],
        max_depth = study.best_trial.params['max_depth'],
        random_strength = study.best_trial.params['random_strength'],
        max_bin = study.best_trial.params['max_bin'],
        od_type = 'Iter',
        one_hot_max_size = study.best_trial.params['one_hot_max_size'], 
        task_type = "GPU")

    optuna_cat.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False, cat_features = cat_cols3)

    cat_val[i_val, :] = optuna_cat.predict_proba(X_train.loc[i_val, :])
    cat_test += optuna_cat.predict_proba(X_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [58]:
print(cat_val.shape)
print(cat_test.shape)

cat_pred = cat_val[:, 1]
cat_pred2 = np.where(cat_pred >= 0.5, 1, 0)
print(f1_score(y_train, cat_pred2))


(501951, 2)
(46404, 2)
0.6642120851585805


In [59]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = cat_pred
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.7031457002729218


In [60]:
# 저장

cat_val_file = pd.DataFrame(cat_val, columns = ['0', '1'])
# cat_part_test_file = pd.DataFrame(cat_part_test, columns = ['0', '1'])
cat_test_file = pd.DataFrame(cat_test, columns = ['0', '1'])
cat_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/cat_val.csv', index = False)
# cat_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/cat_part_test.csv', index = False)
cat_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/cat_test.csv', index = False)

### 6) HGBM

In [61]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_iter': 1000,
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'max_bins': trial.suggest_int('max_bins', 100, 255),
    #    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 20, 80)
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

    model = HistGradientBoostingClassifier(**params_cat)
    model.fit(X_part_train, y_part_train)

    hgbm_pred = model.predict(X_val)
    f1 = f1_score(y_val, hgbm_pred)
    
    return f1

In [62]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "hgbm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-01-27 05:11:38,807] A new study created in memory with name: hgbm_parameter_opt
[I 2022-01-27 05:13:26,402] Trial 0 finished with value: 0.6381335448296699 and parameters: {'learning_rate': 0.012410186619312995, 'max_depth': 25, 'max_bins': 227, 'max_leaf_nodes': 27}. Best is trial 0 with value: 0.6381335448296699.
[I 2022-01-27 05:14:56,123] Trial 1 finished with value: 0.6435684725712882 and parameters: {'learning_rate': 0.015995728111842068, 'max_depth': 17, 'max_bins': 172, 'max_leaf_nodes': 48}. Best is trial 1 with value: 0.6435684725712882.
[I 2022-01-27 05:16:37,590] Trial 2 finished with value: 0.6444116701646907 and parameters: {'learning_rate': 0.018425771696340958, 'max_depth': 28, 'max_bins': 227, 'max_leaf_nodes': 77}. Best is trial 2 with value: 0.6444116701646907.
[I 2022-01-27 05:17:22,227] Trial 3 finished with value: 0.6431912385443206 and parameters: {'learning_rate': 0.06905104955657744, 'max_depth': 14, 'max_bins': 228, 'max_leaf_nodes': 77}. Best is trial

In [63]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6487578228712307
Best trial : {'learning_rate': 0.06857376505098844, 'max_depth': 18, 'max_bins': 189, 'max_leaf_nodes': 51}


In [64]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [65]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [66]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [67]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

hgbm_val = np.zeros((X_train.shape[0], 2))
# hgbm_part_test = np.zeros((X_part_test.shape[0], 2))
hgbm_test = np.zeros((X_test.shape[0], 2))

print(hgbm_val.shape)
# print(hgbm_part_test.shape)
print(hgbm_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_hgbm = HistGradientBoostingClassifier(
        random_state = 39,
        learning_rate = study.best_trial.params['learning_rate'],
        max_depth = study.best_trial.params['max_depth'],
        max_bins = study.best_trial.params['max_bins'],
  #      min_samples_leaf = study.best_trial.params['min_samples_leaf'], 
        max_leaf_nodes = study.best_trial.params['max_leaf_nodes'])

    optuna_hgbm.fit(X_train.loc[i_trn, :], y_train[i_trn])

    hgbm_val[i_val, :] = optuna_hgbm.predict_proba(X_train.loc[i_val, :])
  #  hgbm_part_test += optuna_hgbm.predict_proba(X_part_test) / n_fold
    hgbm_test += optuna_hgbm.predict_proba(X_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [70]:
print(hgbm_val.shape)
#print(hgbm_part_test.shape)
print(hgbm_test.shape)

hgbm_pred = hgbm_val[:, 1]
hgbm_pred2 = np.where(hgbm_pred >= 0.5, 1, 0)
print(f1_score(y_train, hgbm_pred2))

#hgbm_pred2 = hgbm_part_test[:, 1]
#hgbm_pred2 = np.where(hgbm_pred2 >= 0.5, 1, 0)
#print(f1_score(y_part_test, hgbm_pred2))

(501951, 2)
(46404, 2)
0.6373961611522214


In [71]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = hgbm_pred
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6880055917028999


In [72]:
# 저장

hgbm_val_file = pd.DataFrame(hgbm_val, columns = ['0', '1'])
#hgbm_part_test_file = pd.DataFrame(hgbm_part_test, columns = ['0', '1'])
hgbm_test_file = pd.DataFrame(hgbm_test, columns = ['0', '1'])
hgbm_val_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/hgbm_val.csv', index = False)
#hgbm_part_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/hgbm_part_test.csv', index = False)
hgbm_test_file.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/hgbm_test.csv', index = False)

# 5. 2차 모델 생성 (lightgbm)

In [10]:
path = Path('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/')

model_val = ['rf_val', 'lgbm_val', 'xgb_val', 'etc_val', 'cat_val', 'hgbm_val']
# model_part_test = ['rf_part_test', 'lgbm_part_test', 'xgb_part_test', 'etc_part_test', 'cat_part_test', 'hgbm_part_test', 'tab_part_test']
model_test = ['rf_test', 'lgbm_test', 'xgb_test', 'etc_test', 'cat_test', 'hgbm_test']

stk_train = []
# stk_part_test = []
stk_test = []
feature_names1 = []
# feature_names2 = []
feature_names3 = []
for model in model_val :
    stk_train.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names1 += [f'{model}_target0', f'{model}_target1']

# for model in model_part_test :
#     stk_part_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
#     feature_names2 += [f'{model}_target0', f'{model}_target1']

for model in model_test :
    stk_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names3 += [f'{model}_target0', f'{model}_target1']
    
stk_train = np.hstack(stk_train)
# stk_part_test = np.hstack(stk_part_test)
stk_test = np.hstack(stk_test)
print(feature_names1)
# print(feature_names2)
print(feature_names3)
print('456972 =', stk_train.shape)
# print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)
stk_train = stk_train[1 :]
# stk_part_test = stk_part_test[1 :]
stk_test = stk_test[1 :]
print('456972 =', stk_train.shape)
# print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)

KeyboardInterrupt: ignored

In [ ]:
print(y_train.shape)
# print(y_part_test.shape)

In [91]:
def objective(trial : Trial) -> float :

    params_lgb = {
        "random_state": 39,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 40),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 30),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500)
    }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(stk_train, y_train, test_size = 0.2)

    model = LGBMClassifier(**params_lgb)
    model.fit(X_part_train, y_part_train, eval_set = [(X_part_train, y_part_train), (X_val, y_val)],
              early_stopping_rounds = 100, verbose = False)

    stk_lgbm_pred = model.predict(X_val)
    f1 = f1_score(y_val, stk_lgbm_pred)
    
    return f1

In [92]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "stk_lgbm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 500)

[I 2022-01-27 10:28:25,861] A new study created in memory with name: stk_lgbm_parameter_opt
[I 2022-01-27 10:28:32,056] Trial 0 finished with value: 0.6757535332443979 and parameters: {'learning_rate': 0.16859785528215288, 'n_estimators': 818, 'reg_alpha': 8.204018800779794, 'reg_lambda': 1.2204986770815125, 'max_depth': 13, 'num_leaves': 22, 'colsample_bytree': 0.6783450434860411, 'subsample': 0.630012021097161, 'subsample_freq': 19, 'min_child_samples': 93, 'max_bin': 445}. Best is trial 0 with value: 0.6757535332443979.
[I 2022-01-27 10:28:36,141] Trial 1 finished with value: 0.6765551804076606 and parameters: {'learning_rate': 0.2838881081911679, 'n_estimators': 928, 'reg_alpha': 4.148307969361902, 'reg_lambda': 8.258119624773384, 'max_depth': 20, 'num_leaves': 26, 'colsample_bytree': 0.5513618754177441, 'subsample': 0.7146456319775294, 'subsample_freq': 26, 'min_child_samples': 9, 'max_bin': 457}. Best is trial 1 with value: 0.6765551804076606.
[I 2022-01-27 10:28:40,723] Trial 2 

In [ ]:
# Best Score : 0.6286938469013649
# Best trial : {'n_estimators': 729, 
# 'reg_alpha': 0.17560832960220718, 
# 'reg_lambda': 6.7752961532908795, 
# 'max_depth': 4, 
# 'num_leaves': 58, 
# 'colsample_bytree': 0.8287123499658561, 
# 'subsample': 0.4570695128556688, 
# 'subsample_freq': 7, 
# 'min_child_samples': 14, 
# 'max_bin': 207}

In [93]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6829207541385653
Best trial : {'learning_rate': 0.17285728441638123, 'n_estimators': 909, 'reg_alpha': 9.873442300799748, 'reg_lambda': 7.344215235472288, 'max_depth': 11, 'num_leaves': 38, 'colsample_bytree': 0.5352081313855168, 'subsample': 0.8785666281394172, 'subsample_freq': 30, 'min_child_samples': 89, 'max_bin': 246}


In [94]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [95]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [96]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [97]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

stk_lgbm_val2 = np.zeros((stk_train.shape[0], 2))
# stk_lgbm_part_test2 = np.zeros((stk_part_test.shape[0], 2))
stk_lgbm_test2 = np.zeros((stk_test.shape[0], 2))

print(stk_lgbm_val2.shape)
#print(stk_lgbm_part_test2.shape)
print(stk_lgbm_test2.shape)

for i, (i_trn, i_val) in enumerate(cv.split(stk_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_stk_lgbm2 = LGBMClassifier(
                        random_state = 39,
                        n_estimators = study.best_trial.params['n_estimators'], 
                        reg_alpha = study.best_trial.params['reg_alpha'], 
                        reg_lambda = study.best_trial.params['reg_lambda'], 
                        max_depth = study.best_trial.params['max_depth'], 
                        num_leaves = study.best_trial.params['num_leaves'], 
                        colsample_bytree = study.best_trial.params['colsample_bytree'], 
                        subsample = study.best_trial.params['subsample'], 
                        subsample_freq = study.best_trial.params['subsample_freq'], 
                        min_child_samples = study.best_trial.params['min_child_samples'], 
                        max_bin = study.best_trial.params['max_bin'],
                        verbose = 100)

    optuna_stk_lgbm2.fit(stk_train[i_trn], y_train[i_trn], eval_set = [(stk_train[i_val], y_train[i_val])], early_stopping_rounds = 100, eval_metric = 'auc')

    stk_lgbm_val2[i_val, :] = optuna_stk_lgbm2.predict_proba(stk_train[i_val])
    # stk_lgbm_part_test2 += optuna_stk_lgbm2.predict_proba(stk_part_test) / n_fold
    stk_lgbm_test2 += optuna_stk_lgbm2.predict_proba(stk_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
[1]	valid_0's auc: 0.723952	valid_0's binary_logloss: 0.678628
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.725363	valid_0's binary_logloss: 0.666951
[3]	valid_0's auc: 0.726128	valid_0's binary_logloss: 0.657145
[4]	valid_0's auc: 0.726195	valid_0's binary_logloss: 0.649252
[5]	valid_0's auc: 0.726657	valid_0's binary_logloss: 0.642477
[6]	valid_0's auc: 0.727036	valid_0's binary_logloss: 0.638053
[7]	valid_0's auc: 0.727307	valid_0's binary_logloss: 0.633127
[8]	valid_0's auc: 0.726987	valid_0's binary_logloss: 0.63013
[9]	valid_0's auc: 0.727379	valid_0's binary_logloss: 0.626478
[10]	valid_0's auc: 0.727154	valid_0's binary_logloss: 0.624263
[11]	valid_0's auc: 0.72689	valid_0's binary_logloss: 0.622408
[12]	valid_0's auc: 0.726492	valid_0's binary_logloss: 0.620873
[13]	valid_0's auc: 0.727252	valid_0's binary_logloss: 0.618589
[14]	valid_0's auc: 0.726996	valid_0's binary_logloss: 0.617477
[15]

In [98]:
print(stk_lgbm_val2.shape)
# print(stk_lgbm_part_test2.shape)
print(stk_lgbm_test2.shape)

(501951, 2)
(46404, 2)


In [99]:
scores = np.zeros((100, 2))

for i in range(0, 100) :
    threshold = 0.3 + i / 1000
    pred1 = stk_lgbm_val2[:, 1]
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.7098719444932935


In [100]:
scores[scores[:, 1] == max(scores[:, 1])]

array([[0.359     , 0.70987194]])

In [101]:
threshold = 0.359

pred_list = []
scores = []

pred3 = stk_lgbm_test2[:, 1]
pred3 = np.where(pred3 >= threshold , 1, 0)

In [102]:
# 제출

submission_preds = pred3
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/sample_submission.csv')
submission['target'] = submission_preds
submission.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/submission_stk_lgbm_wj_최종.csv', index = False)

# 6. 2차 모델 생성 (catboost)

In [76]:
path = Path('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/')

model_val = ['rf_val', 'lgbm_val', 'xgb_val', 'etc_val', 'cat_val', 'hgbm_val']
# model_part_test = ['rf_part_test', 'lgbm_part_test', 'xgb_part_test', 'etc_part_test', 'cat_part_test', 'hgbm_part_test']
model_test = ['rf_test', 'lgbm_test', 'xgb_test', 'etc_test', 'cat_test', 'hgbm_test']

stk_train = []
# stk_part_test = []
stk_test = []
feature_names1 = []
# feature_names2 = []
feature_names3 = []
for model in model_val :
    stk_train.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names1 += [f'{model}_target0', f'{model}_target1']

# for model in model_part_test :
#     stk_part_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
#     feature_names2 += [f'{model}_target0', f'{model}_target1']

for model in model_test :
    stk_test.append(np.loadtxt(path / f'{model}.csv', delimiter = ','))
    feature_names3 += [f'{model}_target0', f'{model}_target1']
    
stk_train = np.hstack(stk_train)
# stk_part_test = np.hstack(stk_part_test)
stk_test = np.hstack(stk_test)
print(feature_names1)
# print(feature_names2)
print(feature_names3)
print('501951 =', stk_train.shape)
# print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)
stk_train = stk_train[1 :]
# stk_part_test = stk_part_test[1 :]
stk_test = stk_test[1 :]
print('501951 =', stk_train.shape)
# print('44979 =', stk_part_test.shape)
print('46404 =', stk_test.shape)

['rf_val_target0', 'rf_val_target1', 'lgbm_val_target0', 'lgbm_val_target1', 'xgb_val_target0', 'xgb_val_target1', 'etc_val_target0', 'etc_val_target1', 'cat_val_target0', 'cat_val_target1', 'hgbm_val_target0', 'hgbm_val_target1']
['rf_test_target0', 'rf_test_target1', 'lgbm_test_target0', 'lgbm_test_target1', 'xgb_test_target0', 'xgb_test_target1', 'etc_test_target0', 'etc_test_target1', 'cat_test_target0', 'cat_test_target1', 'hgbm_test_target0', 'hgbm_test_target1']
501951 = (501952, 12)
46404 = (46405, 12)
501951 = (501951, 12)
46404 = (46404, 12)


In [60]:
print(y_train.shape)
# print(y_part_test.shape)

(501951,)


In [61]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
  #      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 5, 50.00),
        "n_estimators":trial.suggest_int("n_estimators", 100, 1000),
 #       "max_depth":trial.suggest_int("max_depth", 8, 16),
 #       'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': 'Iter',
  #       'one_hot_max_size' : trial.suggest_int('one_hot_max_size', 0, 5)
  }
    
    X_part_train, X_val, y_part_train, y_val = train_test_split(stk_train, y_train, test_size = 0.2)

    model = CatBoostClassifier(**params_cat,  task_type = "GPU")
    model.fit(X_part_train, y_part_train, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, verbose = False)

    stk_cat_pred = model.predict(X_val)
    f1 = f1_score(y_val, stk_cat_pred)
    
    return f1

In [67]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "stk_cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 500)

[I 2022-01-27 12:50:14,055] A new study created in memory with name: stk_cat_parameter_opt
[I 2022-01-27 12:50:23,686] Trial 0 finished with value: 0.6778571291073938 and parameters: {'bagging_temperature': 17.61404739072837, 'n_estimators': 818, 'max_bin': 446}. Best is trial 0 with value: 0.6778571291073938.
[I 2022-01-27 12:50:31,236] Trial 1 finished with value: 0.6770804187370351 and parameters: {'bagging_temperature': 6.622468041449341, 'n_estimators': 642, 'max_bin': 358}. Best is trial 0 with value: 0.6778571291073938.
[I 2022-01-27 12:50:37,726] Trial 2 finished with value: 0.6803518498978407 and parameters: {'bagging_temperature': 14.550516513062824, 'n_estimators': 524, 'max_bin': 390}. Best is trial 2 with value: 0.6803518498978407.
[I 2022-01-27 12:50:41,896] Trial 3 finished with value: 0.6258942862101549 and parameters: {'bagging_temperature': 42.13411582775548, 'n_estimators': 834, 'max_bin': 484}. Best is trial 2 with value: 0.6803518498978407.
[I 2022-01-27 12:50:45,3

In [68]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.6880127603955132
Best trial : {'bagging_temperature': 46.35729272230932, 'n_estimators': 869, 'max_bin': 303}


In [69]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [70]:
# 파라미터들과의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [71]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [83]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

stk_cat_val = np.zeros((stk_train.shape[0], 2))
# stk_cat_part_test = np.zeros((stk_part_test.shape[0], 2))
stk_cat_test = np.zeros((stk_test.shape[0], 2))

print(stk_cat_val.shape)
# print(stk_cat_part_test.shape)
print(stk_cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(stk_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_stk_cat = CatBoostClassifier(
                        random_state = 39,
   #                      learning_rate = study.best_trial.params['learning_rate'], 
                        bagging_temperature = study.best_trial.params['bagging_temperature'], 
                        n_estimators = study.best_trial.params['n_estimators'], 
   #                      max_depth = study.best_trial.params['max_depth'], 
    #                     random_strength = study.best_trial.params['random_strength'], 
                        max_bin = study.best_trial.params['max_bin'], 
    #                     one_hot_max_size = study.best_trial.params['one_hot_max_size'],
                        task_type = "GPU")

    optuna_stk_cat.fit(stk_train[i_trn], y_train[i_trn], eval_set = [(stk_train[i_val], y_train[i_val])], verbose = False)

    stk_cat_val[i_val, :] = optuna_stk_cat.predict_proba(stk_train[i_val])
    # stk_cat_part_test += optuna_stk_cat.predict_proba(stk_part_test) / n_fold
    stk_cat_test += optuna_stk_cat.predict_proba(stk_test) / n_fold

(501951, 2)
(46404, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [84]:
print(stk_cat_val.shape)
# print(stk_cat_part_test.shape)
print(stk_cat_test.shape)

(501951, 2)
(46404, 2)


In [86]:
scores = np.zeros((300, 2))

for i in range(0, 300) :
    threshold = 0.2 + i / 1000
    pred1 = stk_cat_val[:, 1]
    pred1 = np.where(pred1 >= threshold, 1, 0)
    score = f1_score(y_train, pred1)
    scores[i, 0] = threshold
    scores[i, 1] = score

print(max(scores[:, 1]))


0.6848423447811427


In [ ]:
scores[scores[:, 1] == max(scores[:, 1])] # 0.349

In [89]:
threshold = 0.35

pred_list = []
scores = []

pred3 = stk_cat_test[:, 1]
pred3 = np.where(pred3 >= threshold , 1, 0)

In [90]:
# 제출

submission_preds = pred3
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/sample_submission.csv')
submission['target'] = submission_preds
submission.to_csv('/content/drive/My Drive/Colab Notebooks/공모전/잡케어/submission/submission_stk_cat_wj_최종.csv', index = False)